In [9]:
import cv2
import os
from tqdm.auto import tqdm
from pycocotools.mask import toBbox
import numpy as np
from PIL import Image
from detectron2.structures import BoxMode
from mots_tools_io import (
    load_txt,
)  # download and rename from https://github.com/VisualComputingInstitute/mots_tools/blob/master/mots_common/io.py

In [10]:
folder_images_path = '/home/mcv/datasets/KITTI-MOTS/training/image_02/'
folder_txt_path = '/home/mcv/datasets/KITTI-MOTS/instances_txt/'

In [11]:
objs = load_txt(folder_txt_path+'0000.txt')

[<mots_tools_io.SegmentedObject object at 0x7f3a40dd8390>, <mots_tools_io.SegmentedObject object at 0x7f3a41dba390>]


In [17]:
for i, folder_name in tqdm(enumerate(os.listdir(folder_images_path)),desc='Getting kitti dicts'):
    txt_file = folder_txt_path + folder_name + '.txt'
    print(txt_file)
    for j, img_name in tqdm(enumerate(os.listdir(os.path.join(folder_images_path,folder_name)))):
        print(folder_name, img_name)
        objs = load_txt(folder_txt_path+'0000.txt')
        print(objs)
        break
    break

/home/mcv/datasets/KITTI-MOTS/instances_txt/0004.txt


0004 000128.png
{0: [<mots_tools_io.SegmentedObject object at 0x7f3a40d105d0>, <mots_tools_io.SegmentedObject object at 0x7f3a40d10c90>], 1: [<mots_tools_io.SegmentedObject object at 0x7f3a40d10690>, <mots_tools_io.SegmentedObject object at 0x7f3a40d10990>], 2: [<mots_tools_io.SegmentedObject object at 0x7f3a40d93f50>, <mots_tools_io.SegmentedObject object at 0x7f3a40d106d0>], 3: [<mots_tools_io.SegmentedObject object at 0x7f3a40d07450>, <mots_tools_io.SegmentedObject object at 0x7f3a40d07510>], 4: [<mots_tools_io.SegmentedObject object at 0x7f3a40d07310>, <mots_tools_io.SegmentedObject object at 0x7f3a40d070d0>], 5: [<mots_tools_io.SegmentedObject object at 0x7f3a40d07490>], 6: [<mots_tools_io.SegmentedObject object at 0x7f3a40d071d0>], 7: [<mots_tools_io.SegmentedObject object at 0x7f3a40d10e90>], 8: [<mots_tools_io.SegmentedObject object at 0x7f3a40d07210>], 9: [<mots_tools_io.SegmentedObject object at 0x7f3a40d074d0>], 10: [<mots_tools_io.SegmentedObject object at 0x7f3a40d07410>],

In [ ]:
dataset_dicts = []
for idx, v in enumerate(imgs_anns.values()):
    record = {}

    filename = os.path.join(img_dir, v["filename"])
    height, width = cv2.imread(filename).shape[:2]

    record["file_name"] = filename
    record["image_id"] = idx
    record["height"] = height
    record["width"] = width

    annos = v["regions"]
    objs = []
    for _, anno in annos.items():
        assert not anno["region_attributes"]
        anno = anno["shape_attributes"]
        px = anno["all_points_x"]
        py = anno["all_points_y"]
        poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
        poly = [p for x in poly for p in x]

        obj = {
            "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
            "bbox_mode": BoxMode.XYXY_ABS,
            "segmentation": [poly],
            "category_id": 0,
            "iscrowd": 0
        }
        objs.append(obj)
    record["annotations"] = objs
    dataset_dicts.append(record)

In [ ]:
def to_XYXY(bbox):
    bbox[2] += bbox[0]
    bbox[3] += bbox[1]
    return bbox

objs = load_txt(lbl_path)
for img_path in img_dir.iterdir():
    if img_path.suffix not in [".jpg", ".png"]:
        continue
    idx = int(img_path.stem)
    if idx not in objs or len(objs[idx]) == 0:
        continue

    record = {
        "file_name": str(img_path),
        "image_id": idx,
        "height": objs[idx][0].mask["size"][0],
        "width": objs[idx][0].mask["size"][1],
        "annotations": [
            {
                "bbox": to_XYXY(toBbox(obj.mask)).tolist(),
                "bbox_mode": BoxMode.XYXY_ABS,
                "category_id": 2 if obj.class_id == 1 else 0,
                "iscrowd": 0,
            }
            for obj in objs[idx]
            if obj.class_id != 10
        ],
    }

In [7]:
for f, folder in tqdm(enumerate(listdir(folder_images_path))):
    print("--------------------------")
    images_path = join(folder_images_path, folder)
    onlyimages = [ima for ima in listdir(images_path) if isfile(join(images_path, ima))]
    for i, image in enumerate(onlyimages):
        image_path = folder_images_path + folder + "/" + image 
        label_path = folder_label_path + folder + "/" + image
        
        label = np.asarray(Image.open(label_path))
        height,width = label.shape[:]

        record = {}
        record["file_name"] = image_path
        record["image_id"] = f + i
        record["height"] = height
        record["width"] = width
        patterns = list(np.unique(label))[1:-1]
        objs = []
        for pattern in patterns:
            category_id = int(str(pattern)[0])-1 #Class ids in KITTI-MOTS: (car:1) (pedestrian:2)
            mask_coord = np.argwhere(label==pattern)
            x0, y0 = mask_coord.min(axis=0)    
            x1, y1 = mask_coord.max(axis=0)
            print("XYXY: ", x0,y0,x1,y1)
            bbox = [y0, x0, y1, x1]
            obj = {
                "bbox": bbox,
                "bbox_mode":BoxMode.XYXY_ABS,
                "category_id": category_id,
                "iscrowd": 0
            }
            objs.append(obj)
        record["annotations"] = objs
        print(record)


--------------------------
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]


ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
image_path = '/home/mcv/datasets/KITTI-MOTS/training/image_02/0000/000000.png'
label_path = '/home/mcv/datasets/KITTI-MOTS/instances/0000/000000.png'
label = np.asarray(Image.open(label_path))
height,width = label.shape[:]
print(label.shape[:])
record = {}
record["file_name"] = image_path
record["image_id"] = 
record["height"] = height
record["width"] = width

patterns = list(np.unique(label))[1:-1]
objs = []
for pattern in patterns:
    coord = np.argwhere(label==pattern)
    x0, y0 = coord.min(axis=0)    
    x1, y1 = coord.max(axis=0)
    print(x0,y0)
    print(x1,y1)
    bbox = [y0, x0, y1, x1]
    obj = {
        "bbox": bbox,
        "bbox_mode":BoxMode.XYXY_ABS,
        "category_id": int(np.floor(label[coord[0][0]][label[0][1]]/1e3)-1),
        "iscrowd": 0
    }
objs.append(obj)
record["annotations"] = objs

# TRYING COCOTOOLS

In [ ]:
list(np.unique(label))

In [ ]:
with open(txt_path) as t:
    lines = t.readlines()
    #print(lines)
    bb = toBbox(lines)
    '''lines = t.readlines()
    for line in lines:
        print(line)
        print("--------------------------------------")
        if(line[-1]=="\n"):
            line_splitted = line[:-1].split(' ')
        else:
            line_splitted = line.split(' ')
        print(line_splitted)
        rleObjs = line_splitted[5]
        print(rleObjs)
        bb = toBbox(line_splitted)
        break'''